In [58]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
RSEED=50

In [59]:
X_test = pd.read_csv('X_test.csv')

In [60]:
X_train = pd.read_csv('X_train.csv')

In [61]:
y_test = pd.read_csv('y_test.csv')
y_test.head()

,y
0,0
1,0
2,0
3,0
4,0


In [62]:
y_train = pd.read_csv('y_train.csv')

### Importing the Model


In [63]:
from sklearn.ensemble import RandomForestClassifier


### Building the Model

In [177]:
base_model = RandomForestClassifier()

### Fitting the Model

In [178]:
base_model.fit(X_train, y_train)

C:\Users\rjbat\AppData\Local\Temp\ipykernel_9080\1435837889.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  base_model.fit(X_train, y_train)


RandomForestClassifier()

### Testing Accuracy


In [78]:
y_pred = base_model.predict(X_test)


In [79]:
y_test

,y
0,0
1,0
2,0
3,0
4,0
...,...
822,1
823,0
824,0
825,0


In [80]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

accuracy_score(y_test, y_pred)

0.8270858524788391

## Assessing Random Forest Performance - Base Model

In [81]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

accuracy_score(y_test, y_pred)

0.8270858524788391

In [82]:
confusion_matrix(y_test,y_pred)

array([[465,  61],
       [ 82, 219]], dtype=int64)

In [83]:
precision_score(y_test,y_pred)

0.7821428571428571

In [84]:
recall_score(y_test,y_pred)

0.7275747508305648

### Feature Selection

In [201]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier


In [202]:
def embedded_rf_selector(X, y):
    # Your code goes here (Multiple lines)
    rf = RandomForestClassifier(n_estimators=100)
    embedded_rf_selector = SelectFromModel(rf)
    embedded_rf_selector = embedded_rf_selector.fit(X, y)
    embedded_rf_support = embedded_rf_selector.get_support()
    embedded_rf_feature = X.loc[:, embedded_rf_support].columns.tolist()
    # Your code ends here
    return embedded_rf_support, embedded_rf_feature

In [203]:
embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X_train, y_train)
print(embedded_rf_feature)

c:\Users\rjbat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_selection\_from_model.py:357: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


['balance', 'day', 'month', 'duration', 'pdays', 'poutcome']


### Model with Feature Selection

In [204]:
selected_features = ['balance', 'month', 'duration', 'pdays', 'poutcome']

In [205]:
feature_model = base_model.fit(X_train[selected_features], y_train)

C:\Users\rjbat\AppData\Local\Temp\ipykernel_9080\2352175450.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  feature_model = base_model.fit(X_train[selected_features], y_train)


### Testing Accuracy with Feature Selection

In [206]:
y_pred_feature = feature_model.predict(X_test[selected_features])

In [207]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

accuracy_score(y_test, y_pred_feature)

0.8174123337363967

In [208]:
precision_score(y_test, y_pred_feature)

0.7659574468085106

In [209]:
recall_score(y_test, y_pred_feature)

0.717607973421927

In [210]:
confusion_matrix(y_test,y_pred)

array([[466,  60],
       [ 81, 220]], dtype=int64)

### Parameter Tuning

In [211]:
n_estimators = [5,20,50,100] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}


In [212]:
parameter_model = RandomForestClassifier()

In [213]:
from sklearn.model_selection import RandomizedSearchCV
parameter_random_model = RandomizedSearchCV(estimator = parameter_model,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

In [214]:
parameter_random_model.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\rjbat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\rjbat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': [5, 20, 50, 100]},
                   random_state=35, verbose=2)

In [215]:
print ('Random grid: ', random_grid, '\n')
# print the best parameters
print ('Best Parameters: ', parameter_random_model.best_params_, ' \n')

Random grid:  {'n_estimators': [5, 20, 50, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [2, 6, 10], 'min_samples_leaf': [1, 3, 4], 'bootstrap': [True, False]} 

Best Parameters:  {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 90, 'bootstrap': False}  



In [216]:
random_best_parameters = RandomForestClassifier(n_estimators=50, 
                               min_samples_split=2, 
                               min_samples_leaf = 4,
                               max_features = 'auto',
                               max_depth=90,
                               bootstrap=0,
                               n_jobs=-1, verbose = 1)


In [217]:
random_best_parameters.fit(X_train, y_train)

c:\Users\rjbat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
C:\Users\rjbat\AppData\Local\Temp\ipykernel_9080\3801308204.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_best_parameters.fit(X_train, y_train)
c:\Users\rjbat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\_forest.py:425: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel

RandomForestClassifier(bootstrap=0, max_depth=90, max_features='auto',
                       min_samples_leaf=4, n_estimators=50, n_jobs=-1,
                       verbose=1)

In [218]:
y_pred_parameters = random_best_parameters.predict(X_test)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  50 out of  50 | elapsed:    0.0s finished


### Testing Accuracy of Tuned Parameters

In [219]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score

accuracy_score(y_test, y_pred_parameters)

0.8270858524788391

In [220]:
precision_score(y_test, y_pred_parameters)

0.7743055555555556

In [221]:
recall_score(y_test, y_pred_parameters)

0.7408637873754153

In [222]:
confusion_matrix(y_test,y_pred)

array([[466,  60],
       [ 81, 220]], dtype=int64)